In [2]:
import os

path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)

print("Currently working at {}".format(os.getcwd()))

Currently working at /Users/gieunkwak/Data Analytics/random_things/tridge_demo


In [3]:
import pandas as pd
import time

from fredapi import Fred
import json
from datetime import date
from app.utils.fetch_data import *
# from app.utils.db_connection import *

from collections import Counter

In [29]:
options = fred.search('Producer Price Index by Commodity: Farm Products').reset_index()
options['food_name'] = options['title'].str.split(": ").str[-1]
options['observation_start_year'] = options['observation_start'].dt.year
# filter out unwanted indexes
options = options.loc[~options['food_name'].str.contains('DISCONTINUED')]
options = options.loc[options['frequency_short']=='M']
options = options.loc[options['seasonal_adjustment_short']=='NSA']

In [62]:
today = date.today().strftime("%Y-%m-%d")
today

'2022-05-17'

In [68]:
almonds = fred_fred('WPU01190102', observation_start='2000-01-01', observation_end=today)
hay = fred_fred('WPU01810101', observation_start='2000-01-01', observation_end=today)
avo = fred_fred('WPU01110205', observation_start='2000-01-01', observation_end=today)

2022-05-17 21:31:19,393 [app.utils.fetch_data] [INFO] Fetching data from fred: WPU01110205, from 2000-01-01 to 2022-05-17.


In [100]:
fred_fred('WPU01110204', observation_start='2000-01-01', observation_end=today)

2022-05-17 23:00:05,510 [app.utils.fetch_data] [INFO] Fetching data from fred: WPU01110204, from 2000-01-01 to 2022-05-17.


,date,v,code,p_key
0,2000-01-01,NaN,WPU01110204,2000_01_01_WPU01110204
1,2000-02-01,NaN,WPU01110204,2000_02_01_WPU01110204
2,2000-03-01,NaN,WPU01110204,2000_03_01_WPU01110204
3,2000-04-01,NaN,WPU01110204,2000_04_01_WPU01110204
4,2000-05-01,NaN,WPU01110204,2000_05_01_WPU01110204
...,...,...,...,...
254,2021-03-01,NaN,WPU01110204,2021_03_01_WPU01110204
255,2021-04-01,NaN,WPU01110204,2021_04_01_WPU01110204
256,2021-05-01,NaN,WPU01110204,2021_05_01_WPU01110204
257,2021-06-01,247.900,WPU01110204,2021_06_01_WPU01110204


In [72]:
import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def _plot_two_data(plot_type1, plot_settings):
    """
    Create a plotly figure. The input should be a list of list.
    The inner list should be [dataset, x_column, y_column]

    Args:
        plot_type1(str): type of graph. line or bar
        plot_settings(list): [[dataset, x_column, y_column], [dataset, x_column, y_column], ...]

    Returns:
        fig(
    """
    fig = go.Figure()
    fig = make_subplots(specs=[[{"secondary_y": False}]])

    if plot_type1=='line':
        for setting in plot_settings:
            trace = _create_trace(plot_type1, setting[0], setting[1], setting[2])

    # fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(trace1)
    fig.add_trace(trace2, secondary_y=True)

    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ))

    return fig


def _create_trace(plot_type, data, x, y):
    """
    create trace, which will be a part of the plot
    """
    if plot_type == 'bar':
        trace = go.Bar(x=data[x],
                       y=data[y],
                       name=y)
    if plot_type == 'line':
        trace = go.Scatter(x=data[x],
                           y=data[y],
                           name=y)
    return trace


In [70]:
almonds_trace = _create_trace('line', almonds, 'date', 'v')
hay_trace = _create_trace('line', hay, 'date', 'v')
avo_trace = _create_trace('line', avo, 'date', 'v')

In [83]:
# fig = go.Figure()
fig = make_subplots(specs=[[{"secondary_y": False}]])

fig.add_trace(almonds_trace)
fig.add_trace(hay_trace)
fig.add_trace(avo_trace)
fig

In [84]:
fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ))